In [581]:
import pandas as pd
import numpy as np

In [582]:
data = pd.read_csv("./dataset/news.csv")

In [583]:
data.head()

,id,headline,text
0,uid-1,Market Advances 5.12 More,NEW YORK (AP) - A prime rate reduction by New ...
1,uid-2,District Boosts Request For Anti-Terrorism Aid...,Mayor Anthony A. Williams petitioned the White...
2,uid-3,"Election? Here's How You Do It, Mate.",From our downunder perspective here in Austral...
3,uid-4,The Biggest Boom Ever,We are about to rewrite history. Unless a rece...
4,uid-5,Economic Aide Sees Uptrend,"Sedate and scholarly Dr. Arthur Burns, the ex-..."


In [584]:
data.shape

(3000, 3)

In [585]:
# data.headline.value_counts()

In [586]:
data.isnull().sum()

id          0
headline    0
text        0
dtype: int64

In [587]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()

In [588]:
# data["headline"] = le.fit_transform(data["headline"])
# data.head()

In [589]:
# data.headline.nunique()

In [590]:
data["text_feature"] = data["headline"] + data["text"]

In [591]:
# data["text_feature"] = data["headline"]

In [592]:
import re
def filter_letters(raw_data):
    #data = re.sub("[^a-zA-Z]", ' ', raw_data)
    data = re.sub("[0-9]", ' ', raw_data)
    return data

In [593]:
data["text_feature"] = data["text_feature"].apply(filter_letters)

In [594]:
list = []
for txt in data.text_feature:
    list.append(len(txt))
    

In [595]:
max(list)

5050

In [596]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features= 5050, ngram_range= (1,2), stop_words = 'english', min_df= 1, max_df = 1.0,\
                                   norm = 'l2', sublinear_tf = True)

In [597]:
text_vec = vectorizer.fit_transform(data.text_feature)

In [598]:
indices = np.argsort(vectorizer.idf_)[::-1]

In [599]:
features_text = vectorizer.get_feature_names()
top_n = 10
top_features = [features_text[i] for i in indices[:top_n]]
print(top_features[0:10])

['tel', 'carbon', 'breast', 'fur', 'bcci', 'merck', 'surgical', 'rupee', 'welch', 'altman']


In [600]:
# np.savetxt('tf1.txt', text_vec.toarray(), delimiter=' ')

In [601]:
# text_vect_arr = text_vec.toarray()

In [602]:
# text_vect_arr[0:10]

In [603]:
# vectorizer.get_feature_names()

In [604]:
text_dataframe = pd.DataFrame(text_vec.toarray(), columns=vectorizer.get_feature_names())
text_dataframe.head()

,abandoned,abc,ability,able,abroad,absence,absolutely,abuse,academic,accelerate,...,york nov,york oct,york sept,york stock,york trading,young,younger,yuan,zero,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [605]:
text_dataframe1 = text_dataframe.loc[:,top_features]

In [606]:
features = pd.concat([data, text_dataframe1], axis = 1)

In [607]:
features = features.drop(["text", "text_feature", "headline"], axis = 1)

In [608]:
features.shape

(3000, 11)

In [609]:
features = features.set_index("id")

In [610]:
features.head()

,tel,carbon,breast,fur,bcci,merck,surgical,rupee,welch,altman
id,,,,,,,,,,
uid-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uid-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uid-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uid-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uid-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [611]:
np.savetxt('tfm5.txt', features.as_matrix())

In [612]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [613]:
# for k in [2, 10, 50, 100, 200, 500, 1000]:
    
#     clusterer = GaussianMixture(n_components= k).fit(features)
    
#     preds = clusterer.predict(features)
    
#     print(silhouette_score(features, preds, random_state=0))

In [614]:
from sklearn.cluster import KMeans
score = {}
for k in range(2,50,2):
    clusterer = KMeans(n_clusters=k, random_state=100).fit(features)
    preds = clusterer.predict(features)
    score.update({k:silhouette_score(features, preds, random_state=0)})
#     sc = silhouette_score(features, preds, random_state=0)
#     score.append(sc)
    print(k,silhouette_score(features, preds, random_state=0))
#     features["cluster"] = preds
#     features["cluster"].to_csv("cluster_result.csv")
# print(score, np.argmax(score))
print(score)

2 0.989734504106
4 0.990273819085
6 0.992052992316
8 0.992659475304
10 0.992878128462
12 0.993260423182
14 0.994639012408
16 0.995046431996
18 0.995417990686
20 0.995173917985
22 0.994878161984
24 0.994266852903
26 0.993334775782
28 0.99217009858
30 0.990981534414
32 0.989666666667
34 0.989666666667
36 0.989666666667
38 0.989666666667
40 0.989666666667
42 0.989666666667
44 0.989666666667
46 0.989666666667
48 0.989666666667
{2: 0.98973450410571062, 4: 0.99027381908491496, 6: 0.99205299231573218, 8: 0.99265947530392484, 10: 0.99287812846205847, 12: 0.9932604231819, 14: 0.99463901240757435, 16: 0.99504643199649245, 18: 0.99541799068603121, 20: 0.99517391798537969, 22: 0.99487816198379553, 24: 0.9942668529034987, 26: 0.99333477578207063, 28: 0.99217009858012839, 30: 0.99098153441404946, 32: 0.98966666666666669, 34: 0.98966666666666669, 36: 0.98966666666666669, 38: 0.98966666666666669, 40: 0.98966666666666669, 42: 0.98966666666666669, 44: 0.98966666666666669, 46: 0.98966666666666669, 48: 0.

In [615]:
sorted(score.items(), key=lambda x: x[1])

[(32, 0.98966666666666669),
 (34, 0.98966666666666669),
 (36, 0.98966666666666669),
 (38, 0.98966666666666669),
 (40, 0.98966666666666669),
 (42, 0.98966666666666669),
 (44, 0.98966666666666669),
 (46, 0.98966666666666669),
 (48, 0.98966666666666669),
 (2, 0.98973450410571062),
 (4, 0.99027381908491496),
 (30, 0.99098153441404946),
 (6, 0.99205299231573218),
 (28, 0.99217009858012839),
 (8, 0.99265947530392484),
 (10, 0.99287812846205847),
 (12, 0.9932604231819),
 (26, 0.99333477578207063),
 (24, 0.9942668529034987),
 (14, 0.99463901240757435),
 (22, 0.99487816198379553),
 (16, 0.99504643199649245),
 (20, 0.99517391798537969),
 (18, 0.99541799068603121)]

In [620]:
from sklearn.cluster import KMeans
clusterer = KMeans(n_clusters=18, random_state=400).fit(features)
preds = clusterer.predict(features)
print(silhouette_score(features, preds, random_state=0))
features["cluster"] = preds
# features["cluster"].to_csv("cluster_result1.csv")

0.997266664339


In [578]:
features = features.reset_index()
features.head()

,id,wholesale,income,supply,care,selling,chairman,imf,benefits,action,lift,cluster
0,uid-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,uid-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,uid-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,uid-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,uid-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [579]:
features.to_csv("cluster_result2.csv", columns = ["id", "cluster"], index= False)

In [580]:
features.head()

,id,wholesale,income,supply,care,selling,chairman,imf,benefits,action,lift,cluster
0,uid-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,uid-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,uid-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,uid-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,uid-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [494]:
# df1 = features[0:10]

In [495]:
# df1